All right, for our next demo let's do some sentiment analysis on this restaurant data. I suppose we'll use rating as the target.

In [1]:
import pandas as pd
import numpy as np

import regex as re
from nltk.stem.porter import PorterStemmer

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../data/Restaurant reviews.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '') #Clean the titles for ease of coding
print(df.shape)
df.head()

(10000, 8)


,restaurant,reviewer,review,rating,metadata,time,pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [3]:
df.isnull().sum()

restaurant       0
reviewer        38
review          45
rating          38
metadata        38
time            38
pictures         0
7514          9999
dtype: int64

Thankfully we have the restaurant - as it'd be essentially impossible to do any type of data subsitution for that.<br>
Reviewer could be interesting if we only have like 10 food critics and can actually observe how each one words their dis/likes. However, I'd say if it's very large we'll just ignore it for our purposes (which is just to demonstrate potential business applications of NLP/sentiment analysis).<br>
Review - what we care about. Enough said. Of note will be if there's potentially a trend for more positive/negative reviews to be wordier (using the derived word count as a potential feature).<br>
Rating - enough said.<br>
Metadata - some type of social media thing. May contribute to potentially being a more positive or negative review - to garner more of a reaction.<br>
Time - what to say when it comes to potential traffic, holidays, etc.. However, I doubt I'll get into that with this project.<br>
Picture - seeming a binary to suggest if the person took it, likely a sign of emotion either way. <br>
<br><br> And now:

In [4]:
df.drop('7514', axis=1, inplace=True) #Not only do I have no clue what this is, essentially it's all nulls...

In [5]:
#Let's use our handly feautre analyzer formula again:
#Somewhat adjusted now to just assume if texts (per the below criteria) to merely describe it so the screen doesn't get flooded.

def variable_analyzer(df, col, categorical_norm=True):
    print(f"Now looking at feature {df.columns.get_loc(col)}, {col}.")
    nulls = df[col].isnull().sum()
    if nulls > 0:
        print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
    print('~~~')
    #The firs branch deals with numeric; second categorical.
    if (10 <= len(df[df.columns[i]].value_counts()) and df[df.columns[i]].dtype in ['int64', 'float64']) or (len(df[df.columns[i]].value_counts()) and df[df.columns[i]].dtype == 'O'): #Arb. threshold
        print(df[df.columns[i]].describe()) #note that display could be another option.
    else:
        print(f"{df.columns[i]} has {df[df.columns[i]].value_counts().count()} categories.")
        print(df[df.columns[i]].value_counts(normalize=categorical_norm))

In [6]:
i = -1

In [7]:
i += 1
variable_analyzer(df, df.columns[i])

Now looking at feature 0, restaurant.
~~~
count               10000
unique                100
top       Beyond Flavours
freq                  100
Name: restaurant, dtype: object


Oh, I didn't know that - an even 100 restaurants. Might be worthwhile to group then each one's reviews separately. As, perhaps one's 'good' is another's 'mediocre'. Regardless, we won't be that specific and continue, mushing them all together (into one giant cholent).

In [8]:
i += 1
variable_analyzer(df, df.columns[i])

Now looking at feature 1, reviewer.
Reminder that reviewer has 38 nulls - deal with them noob.
~~~
count            9962
unique           7446
top       Parijat Ray
freq               13
Name: reviewer, dtype: object


In [9]:
df['reviewer'].describe()

count            9962
unique           7446
top       Parijat Ray
freq               13
Name: reviewer, dtype: object

Hmm, with the top reviewer only having 13 reviews I won't bother doing anything with reviewer (to potentially analyze their own persoanl usage of whatever words).

In [10]:
i += 1
variable_analyzer(df, df.columns[i])

Now looking at feature 2, review.
Reminder that review has 45 nulls - deal with them noob.
~~~
count     9955
unique    9364
top       good
freq       237
Name: review, dtype: object


In [11]:
j =-1

In [12]:
j += 1
print(f"Currently looking at the {j} indexed review")
df['review'][j] #To look at least a few of them (change code as desired())

Currently looking at the 0 indexed review


'The ambience was good, food was quite good . had Saturday lunch , which was cost effective .\nGood place for a sate brunch. One can also chill with friends and or parents.\nWaiter Soumen Das was really courteous and helpful.'

In [13]:
i += 1
variable_analyzer(df, df.columns[i])

Now looking at feature 3, rating.
Reminder that rating has 38 nulls - deal with them noob.
~~~
count     9962
unique      10
top          5
freq      3832
Name: rating, dtype: object


Odd, it's showing up as an object. Let's clean this up (data entry stuff as sometimes numbers get recorded as 'strings', like a letter, making it unable to do mathematical observations about it.)<br>
Depending upon the context (ie excellent isn't exactly '5', with the numbers conveying more of a sliding scale), it might be worthwhile to categorize them. However, without any additional knowledge I won't assume so.<br><br>
See below though as it seems what's causing them to be considered an object is that one like. And hence, the first dilemma - what to do?<br>Well, let's start with actually reading it:

In [22]:
df['rating'].value_counts()

rating
5      3832
4      2373
1      1735
3      1193
2       684
4.5      69
3.5      47
2.5      19
1.5       9
5         1
Name: count, dtype: int64

In [15]:
review_row = df[df['rating']=='Like'].index[0] #To make referencing it easier
df.loc[review_row, 'review']

This is quite a positive review. However, I'm unsure if this person would rate this a 4, 4.5, or 5?<br><br>In a regular work scenario I would likely drop it as we have more than enough data to work with, however for the same of my demo I will replace this with a 5 (being the dominant class of the three options I would narrow this down to) purely for a vain reason - I want to say I workd with 10k records (I might be talking about this a lot so I'd rather make an even 10k).

In [18]:
df.loc[review_row, 'rating'] = 5

I still need to deal with the nulls now too. Eh, see above as I'll also fill these in. Since we have 38 I'll use the median, a typical choice for filling null values. <br>Hmm, upon cleeaning the data I confirmed that the data was stored as a string. Let's take care of that too.

In [27]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [31]:
print(f"The median rating is {df['rating'].median()}.")
df['rating'].fillna(df['rating'].median(), inplace=True)

The median rating is 4.0.


In [33]:
#Now to propely see and comment on this.
variable_analyzer(df, df.columns[i])

Now looking at feature 3, rating.
~~~
rating has 9 categories.
rating
5.0    0.3833
4.0    0.2411
1.0    0.1735
3.0    0.1193
2.0    0.0684
4.5    0.0069
3.5    0.0047
2.5    0.0019
1.5    0.0009
Name: proportion, dtype: float64


In [35]:
df['rating'].describe() #Per how many quick-glance code is I need to manually do this. So too for anything else a bit more
#peculiar, like metadata coming up.

count    10000.000000
mean         3.602700
std          1.480835
min          1.000000
25%          3.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: rating, dtype: float64

Interesting. So we see an ovrwhelming majority of positive reviews (appox 62%) with negatives (2 and below) being about 23%. Let's see how accurately our sentiment will be able to pick up on this.

In [36]:
i += 1
variable_analyzer(df, df.columns[i])

Now looking at feature 4, metadata.
Reminder that metadata has 38 nulls - deal with them noob.
~~~
count         9962
unique        2477
top       1 Review
freq           919
Name: metadata, dtype: object


In [38]:
df['metadata'].value_counts()

metadata
1 Review                        919
1 Review , 1 Follower           750
2 Reviews , 1 Follower          337
2 Reviews                       262
1 Review , 2 Followers          227
                               ... 
22 Reviews , 22 Followers         1
14 Reviews , 94 Followers         1
32 Reviews , 1445 Followers       1
33 Reviews , 8 Followers          1
472 Reviews , 1302 Followers      1
Name: count, Length: 2477, dtype: int64

Hmm, this is interesting. This seems to be two different columns shmushed into one wtih a recording of how many food reviews they did and how many people follow them for whatever reason (entirely food-based or not).<br>
For the sake of this sentiment analysis demo I doubt I'll get into it, but at least I'll do the following:

In [43]:
df[df['metadata']=='1 Review']['rating'].value_counts(normalize=True)

rating
5.0    0.528836
1.0    0.266594
4.0    0.078346
2.0    0.067465
3.0    0.056583
4.5    0.001088
2.5    0.001088
Name: proportion, dtype: float64

Hmm. Recall that the plurality of any review was positive. However, this intrigues me - did the food astonsih and inspire and woo them so much that they just made a social media post, their first one (review), just for this? Yeah, this for sure is a fun column. Perhaps we'll go deeper than a simple sentiment analysis. We'll see.<br>I just got excited thinking about separating them then engineering a 'new column' reflecting online-presence, a scalar of reviews and followers.

In [44]:
i += 1
variable_analyzer(df, df.columns[i])

Now looking at feature 5, time.
Reminder that time has 38 nulls - deal with them noob.
~~~
count                9962
unique               9782
top       7/29/2018 20:34
freq                    3
Name: time, dtype: object


Yeah, we're not bothering with this one (see above).

In [45]:
i += 1
variable_analyzer(df, df.columns[i])

Now looking at feature 6, pictures.
~~~
count    10000.000000
mean         0.748600
std          2.570381
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         64.000000
Name: pictures, dtype: float64


In [47]:
df['pictures'].value_counts()[:10]

pictures
0    8016
1     707
2     383
3     246
4     129
5     116
7      86
6      85
8      55
9      47
Name: count, dtype: int64

Likely accompanying the picture to enhance their review/emotionn. We'll look at a few before continuing:

In [50]:
df[df['pictures']==0]['rating'].describe()

count    8016.000000
mean        3.538049
std         1.558263
min         1.000000
25%         2.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

In [51]:
df[df['pictures']>0]['rating'].describe()

count    1984.000000
mean        3.863911
std         1.075973
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

Hmm, there is a higher mean with the more pictures. Likely showing more emotion. I wonder if also correlated to followers (and review).

## Text Clean Up

Well, we did basic EDA. Onto the special stuff that's motivating me to demeo this project.<br><br> Yeah, I'm too interested in the metadata not to look at it. We'll deal with that first as it should be quicker as well as saving our main focus for the end.

In [91]:
#Code I found to much more elegantly clean this up.
def extract_count(text, keyword):
    if not isinstance(text, str):
        return 0
    
    text = text.lower()
    keyword = keyword.lower()

    pattern = fr'(\d+)\s*{re.escape(keyword)}s?'
    match = re.search(pattern, text)
    
    if match:
        return int(match.group(1))
    return None

In [92]:
df['number_of_reviews'] = df['metadata'].apply(lambda x: extract_count(x, 'review'))

In [99]:
df['number_of_reviews'].fillna(0, inplace=True) #Any remaiing nulls, which seemingly come about when review isn't in the text, are
#definitionally 0s. So too below.

In [100]:
df['number_of_reviews'].value_counts()

number_of_reviews
1.0      2480
2.0      1166
3.0       714
4.0       501
5.0       371
         ... 
149.0       1
103.0       1
223.0       1
390.0       1
472.0       1
Name: count, Length: 234, dtype: int64

In [101]:
df['number_of_reviews'].describe()

count    10000.000000
mean        23.853900
std         50.515055
min          0.000000
25%          1.000000
50%          4.000000
75%         22.000000
max       1031.000000
Name: number_of_reviews, dtype: float64

In [112]:
df[df['number_of_reviews']<2]['rating'].describe()

count    2685.000000
mean        3.641713
std         1.712454
min         1.000000
25%         2.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

In [113]:
df[df['number_of_reviews']>=2]['rating'].describe()

count    7315.000000
mean        3.588380
std         1.385997
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

All right, that should be all cleaned up now. Let's finally comment.<br><br>
Generally one review, yet once you get past one it really explodes. Likely a similar tred below. Also, notice at how incredibly skewed to the right the data is with a mean of almost 24; yet a median of only 4!<br>Hmm, this requires some more investigation later as bisecting the ratings at 2 reviws yields a close enough median and close media (as we assumed a 4 is still positive). At some point the casual becomes a critic, hence, it makes sense to expect a lower average as reviews increase.

In [95]:
df['number_of_followers'] = df['metadata'].apply(lambda x: extract_count(x, 'follow'))

In [102]:
df['number_of_followers'].fillna(0, inplace=True)

In [114]:
df['number_of_followers'].value_counts()

number_of_followers
1.0       1634
0.0       1616
2.0        695
3.0        387
4.0        279
          ... 
1600.0       1
1127.0       1
1196.0       1
1453.0       1
423.0        1
Name: count, Length: 751, dtype: int64

In [115]:
df['number_of_followers'].describe()

count    10000.000000
mean       162.374800
std        618.852357
min          0.000000
25%          1.000000
50%          6.000000
75%         63.000000
max      13410.000000
Name: number_of_followers, dtype: float64

In [117]:
df[df['number_of_followers']<2]['rating'].describe()

count    3250.000000
mean        3.490923
std         1.688999
min         1.000000
25%         1.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

In [118]:
df[df['number_of_followers']>=2]['rating'].describe()

count    6750.000000
mean        3.656519
std         1.366247
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

A similar trend to above, with the same difference being the scale (here the proportion from the mean to the median is even more exaggerated: 162 to 6! Ultimtely, this scale makes sense as the online post scales to generally generate multiple followes per published item.

In [105]:
df['number_of_posts'] = df['metadata'].apply(lambda x: extract_count(x, 'post'))

In [120]:
df['number_of_posts'].fillna(0, inplace=True)

In [121]:
df['number_of_posts'].value_counts()

number_of_posts
0.0      9994
1.0         1
89.0        1
82.0        1
151.0       1
47.0        1
88.0        1
Name: count, dtype: int64

In [123]:
df['number_of_posts'].describe()

count    10000.000000
mean         0.045800
std          2.176781
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        151.000000
Name: number_of_posts, dtype: float64

In [127]:
df[df['number_of_posts']==0]['rating'].describe()

count    9994.000000
mean        3.602311
std         1.481165
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

In [128]:
df[df['number_of_posts']>0]['rating'].describe()

count    6.00000
mean     4.25000
std      0.41833
min      4.00000
25%      4.00000
50%      4.00000
75%      4.37500
max      5.00000
Name: rating, dtype: float64

Almost negligible upon actually looking at it, with almost everything being 0/no posts. However, maybe it'll make a difference. Likely when we make the social-presence we'll not include this (as otherwise everything just becomes 0, multiplying it into the mix....).

### Reviews to Features

The time has come.